In [1]:
import numpy as np
import pandas as pd

# Análisis de la Data

El archivo cuenta con un total de 64 columnas. Sin embargo, del análisis de cada columna y la información necesaria para realizar el presente proyecto, sólo vamos a seleccionar 11. 

In [2]:
# Path para acceder al archivo csv
archivo = '../Data/Compras_modificado.csv'

# Selección de columnas
columnas_select = ['Coleccion', 'Referencia', 'Color', 'Descripcion', 'No.Pedido', 
                   'Tot_Piezas', 'Costo_Fabrica', 'Precio_Venta',
                   'Pais_Producto', 'Cta_Cliente', 'Nombre_Cliente', 
                   'Estatus', 'Plataforma', 'familia']

# Abrir archivo csv con columnas seleccionadas
data_df = pd.read_csv(archivo, usecols=columnas_select)

In [4]:
data_df.head()

,Coleccion,Referencia,Descripcion,Color,No.Pedido,Tot_Piezas,Costo_Fabrica,Precio_Venta,Pais_Producto,Cta_Cliente,Nombre_Cliente,Estatus,Plataforma,familia
0,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5590,12.0,15.65,25.0,CHINA,1207097,RESERVA SUCURSALES,EN SISTEMA,ASIA,RK
1,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5595,12.0,15.65,25.0,CHINA,1207097,RESERVA SUCURSALES,EN SISTEMA,ASIA,RK
2,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5597,6.0,15.65,25.0,CHINA,2313076,"RIPOSTO,S.A.",EN SISTEMA,ASIA,RK
3,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5599,4.0,15.65,25.0,CHINA,2305201,"MYL DE COLOMBIA,SAS",EN SISTEMA,ASIA,RK
4,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5600,4.0,15.65,25.0,CHINA,2305201,"MYL DE COLOMBIA,SAS",EN SISTEMA,ASIA,RK


## Análisis general de los datos 

In [5]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65597 entries, 0 to 65596
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Coleccion       65597 non-null  object 
 1   Referencia      65597 non-null  object 
 2   Descripcion     65597 non-null  object 
 3   Color           65597 non-null  object 
 4   No.Pedido       65597 non-null  int64  
 5   Tot_Piezas      65597 non-null  float64
 6   Costo_Fabrica   65597 non-null  float64
 7   Precio_Venta    65597 non-null  float64
 8   Pais_Producto   65597 non-null  object 
 9   Cta_Cliente     65597 non-null  int64  
 10  Nombre_Cliente  65597 non-null  object 
 11  Estatus         65597 non-null  object 
 12  Plataforma      65597 non-null  object 
 13  familia         65597 non-null  object 
dtypes: float64(3), int64(2), object(9)
memory usage: 7.0+ MB


In [6]:
data_df.describe()

,No.Pedido,Tot_Piezas,Costo_Fabrica,Precio_Venta,Cta_Cliente
count,65597.000000,65597.000000,65597.000000,65597.000000,6.559700e+04
mean,7478.672805,17.126500,24.831679,44.497119,2.052870e+06
std,1367.369884,30.368732,12.421134,21.193926,4.587816e+05
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,6405.000000,5.000000,17.470000,33.000000,2.201119e+06
50%,7228.000000,8.000000,26.600000,44.000000,2.224074e+06
75%,8747.000000,15.000000,31.910000,55.000000,2.306225e+06
max,9685.000000,849.000000,148.960000,1000.000000,2.601001e+06


In [7]:
data_df.isnull().sum()

Coleccion         0
Referencia        0
Descripcion       0
Color             0
No.Pedido         0
Tot_Piezas        0
Costo_Fabrica     0
Precio_Venta      0
Pais_Producto     0
Cta_Cliente       0
Nombre_Cliente    0
Estatus           0
Plataforma        0
familia           0
dtype: int64

**No hay datos nulos**

# Pre-procesamiento básico

En este pre-procesamiento básico, se van a realizar los siguientes ajustes:

1. Eliminar duplicados
2. Filtrar columna "Estatus" para eliminar pedidos "cancelados"
3. Filtrar clientes que desde 2019 no compran
4. Verificar existencia de datos nulos. Anteriormente se observó que no hay NaN, pero se verificarán otras formas de existencia de datos no válidos
4. Ajustar errores de texto en columnas con dato tipo obj

## Eliminar duplicados

In [8]:
data_df.shape

(65597, 14)

In [9]:
data_proces_df = data_df.drop_duplicates()

In [10]:
data_proces_df.shape

(65419, 14)

In [11]:
duplicados = data_df.shape[0] - data_proces_df.shape[0]
print(f'Se eliminaron {duplicados} filas duplicadas')

Se eliminaron 178 filas duplicadas


## Filtrado según estatus de la compra

Se eliminan las compras con estatus "cancelado" y "caida"

In [12]:
data_proces_df['Estatus'].unique()

array(['EN SISTEMA', 'CANCELADO', 'CAIDA', 'EN PROCESO',
       '               ', 'EN BODEGA', 'EN PUERTO', '03/22/2024',
       '02/16/2024', '02/23/2024', '02/14/2024', '02/19/2024',
       '03/08/2024', '02/09/2024'], dtype=object)

In [13]:
data_proces_df['Estatus'].value_counts()

EN SISTEMA         45387
CANCELADO          14164
                    4032
CAIDA                490
02/14/2024           452
03/22/2024           280
02/19/2024           182
02/16/2024           154
EN PROCESO           115
03/08/2024            70
EN PUERTO             38
02/23/2024            35
EN BODEGA             10
02/09/2024            10
Name: Estatus, dtype: int64

In [14]:
# Filtrar data set. Eliminamos compras canceladas o caídas
data_proces_status = data_proces_df[(data_proces_df['Estatus'] != 'CANCELADO') & (data_proces_df['Estatus'] != 'CAIDA')]

In [15]:
data_proces_status['Estatus'].unique()

array(['EN SISTEMA', 'EN PROCESO', '               ', 'EN BODEGA',
       'EN PUERTO', '03/22/2024', '02/16/2024', '02/23/2024',
       '02/14/2024', '02/19/2024', '03/08/2024', '02/09/2024'],
      dtype=object)

In [16]:
compras_filtradas = data_proces_status.shape[0]
compras_filtradas

50765

In [17]:
compras_canceladas = data_proces_df.shape[0] - compras_filtradas
print(f'Se filtró la base, se eliminaron {compras_canceladas} compras canceladas o caídas')

Se filtró la base, se eliminaron 14654 compras canceladas o caídas


## Datos nulos y ajuste en texto

Del análisis inicial se observó que no hay datos NAN, pero se analizarán otras formas de datos nulos para como texto vacío, caracteres especiales, etc

In [18]:
columns = data_proces_status.columns
columns

Index(['Coleccion', 'Referencia', 'Descripcion', 'Color', 'No.Pedido',
       'Tot_Piezas', 'Costo_Fabrica', 'Precio_Venta', 'Pais_Producto',
       'Cta_Cliente', 'Nombre_Cliente', 'Estatus', 'Plataforma', 'familia'],
      dtype='object')

In [19]:
range(len(data_proces_status.columns))

range(0, 14)

In [20]:
valores_unicos = {}

for column in columns:
    valores_u = data_proces_status[column].unique()
    valores_unicos[column] = valores_u

In [21]:
valores_unicos

{'Coleccion': array(['AW2016', 'AW2017', 'AW2018', 'AW2019', 'AW2020', 'AW2021',
        'FW2022', 'FW2023', 'FW2024', 'SS2016', 'SS2017', 'SS2018',
        'SS2019', 'SS2020', 'SS2021', 'SS2022', 'SS2023', 'SS2024'],
       dtype=object),
 'Referencia': array(['RK2321', 'RK2447', 'RK2464', ..., 'TJ7737 PE', 'TJ7950 PE',
        'TJ7951 PE'], dtype=object),
 'Descripcion': array(['GORRA LACOSTE PARA CABALLERO', 'MEDIAS LACOSTE PARA CABALLERO',
        'CAMISA PARA CABALLERO', 'JACKET PARA CABALLERO',
        'PANTALON LARGO PARA CABALLERO', 'CAMISA TIPO POLO PARA CABALLERO',
        'BERMUDA PARA CABALLERO', 'PANTALON CORTO PARA CABALLERO',
        'SUETER CON CAPUCHA PARA CABALLERO',
        'T-SHIRT CUELLO REDONDO PARA CABALLERO',
        'T-SHIRT CUELLO V PARA CABALLERO', 'T-SHIRT PARA CABALLERO',
        'SUETER MANGA LARGA CON CAPUCHA PARA CABALLERO',
        'CONJUNTO DEPORTIVO PARA CABALLERO', 'CARDIGAN PARA CABALLERO',
        'PULLOVER PARA CABALLERO', 'SUETER MANGA LARGA PARA

### Ajustes columnas "Pais Productor"

In [22]:
valores_unicos['Pais_Producto']

array(['CHINA', 'RUMANIA', 'VIETNAM', 'CHINA                    00042',
       'MALASIA', 'INDONESIA', 'TUNISIA', 'PERU', 'EL SALVADO', 'FRANCIA',
       'MARRUECOS', 'COREA DEL SUR', 'LITUANIA', 'EL SALVADOR', 'TUNEZ',
       'BULGARIA', 'CAMBODIA', 'PORTUGAL', 'MAURICIO', 'TURQUIA',
       'SRI LANKA', 'COLOMBIA', 'GUATEMALA', 'TAILANDIA', 'MADAGASCAR',
       'CAMBOYA', 'EGIPTO', 'SERBIA', '                              ',
       'PARAGUAY', 'MEXICO', 'ITALIA', 'BIELORUSIA', 'SERVIA'],
      dtype=object)

In [23]:
import string
import re

def basic_text_preprocess(sentence):
    """Función que realiza limpieza de texto. Incluye limpieza básica: eliminar espacios, pasar a minúscula,
    eliminar números y puntuación"""
    sentence = sentence.strip() #elimina espacios
    sentence = sentence.lower() #pasa texto a minúscula
    sentence = ''.join(char for char in sentence if not char.isdigit()) #elimina números

    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') #elimina puntuaciones
        
    # Eliminar espacios duplicados resultantes de las sustituciones
    sentence = re.sub(r'\s+', ' ', sentence).strip()
        
    return sentence

In [24]:
data_proces_status.shape

(50765, 14)

In [25]:
data_proces_status['Pais_Producto'] = data_proces_status['Pais_Producto'].apply(basic_text_preprocess)

/tmp/ipykernel_1064/1779119270.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_proces_status['Pais_Producto'] = data_proces_status['Pais_Producto'].apply(basic_text_preprocess)


In [26]:
data_proces_status.head()

,Coleccion,Referencia,Descripcion,Color,No.Pedido,Tot_Piezas,Costo_Fabrica,Precio_Venta,Pais_Producto,Cta_Cliente,Nombre_Cliente,Estatus,Plataforma,familia
0,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5590,12.0,15.65,25.0,china,1207097,RESERVA SUCURSALES,EN SISTEMA,ASIA,RK
1,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5595,12.0,15.65,25.0,china,1207097,RESERVA SUCURSALES,EN SISTEMA,ASIA,RK
2,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5597,6.0,15.65,25.0,china,2313076,"RIPOSTO,S.A.",EN SISTEMA,ASIA,RK
3,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5599,4.0,15.65,25.0,china,2305201,"MYL DE COLOMBIA,SAS",EN SISTEMA,ASIA,RK
4,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5600,4.0,15.65,25.0,china,2305201,"MYL DE COLOMBIA,SAS",EN SISTEMA,ASIA,RK


In [27]:
data_proces_status['Pais_Producto'].value_counts()

peru             14976
china            13579
tunez             3032
vietnam           2824
turquia           2781
indonesia         2328
sri lanka         1858
cambodia          1173
rumania            969
marruecos          919
colombia           872
mauricio           776
el salvador        736
camboya            698
francia            617
tunisia            507
madagascar         459
malasia            453
paraguay           253
guatemala          237
el salvado         176
lituania            79
bulgaria            78
mexico              73
tailandia           72
serbia              66
egipto              47
                    38
portugal            35
servia              30
corea del sur       19
italia               4
bielorusia           1
Name: Pais_Producto, dtype: int64

In [28]:
def replace_countries(sentence):
    '''Función para corregir errores de tipeo identificado en el análisis de valores únicos'''
    sentence = re.sub(r'\bcambodia\b', "camboya", sentence)
    sentence = re.sub(r'\bel salvado\b', "el salvador", sentence)
    sentence = re.sub(r'\btunisia\b', "tunez", sentence)
    sentence = re.sub(r'\bservia\b', "serbia", sentence)
    
    return sentence

In [29]:
# Aplico función para corregir errores de escritura en columna "Pais Productor"
data_proces_status['Pais_Producto_fil'] = data_proces_status['Pais_Producto'].apply(replace_countries)

/tmp/ipykernel_1064/2290828643.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_proces_status['Pais_Producto_fil'] = data_proces_status['Pais_Producto'].apply(replace_countries)


In [30]:
data_proces_status['Pais_Producto_fil'].value_counts()

peru             14976
china            13579
tunez             3539
vietnam           2824
turquia           2781
indonesia         2328
camboya           1871
sri lanka         1858
rumania            969
marruecos          919
el salvador        912
colombia           872
mauricio           776
francia            617
madagascar         459
malasia            453
paraguay           253
guatemala          237
serbia              96
lituania            79
bulgaria            78
mexico              73
tailandia           72
egipto              47
                    38
portugal            35
corea del sur       19
italia               4
bielorusia           1
Name: Pais_Producto_fil, dtype: int64

In [31]:
# Se observa que hay filas vacías que no son datos nulos, en este caso, se reemplaza por "sin dato"
data_proces_status['Pais_Producto_fil'] = data_proces_status['Pais_Producto_fil'].str.replace(r'^\s*$', 'sin dato', regex=True)

/tmp/ipykernel_1064/1747797357.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_proces_status['Pais_Producto_fil'] = data_proces_status['Pais_Producto_fil'].str.replace(r'^\s*$', 'sin dato', regex=True)


In [32]:
data_proces_status['Pais_Producto_fil'].value_counts()

peru             14976
china            13579
tunez             3539
vietnam           2824
turquia           2781
indonesia         2328
camboya           1871
sri lanka         1858
rumania            969
marruecos          919
el salvador        912
colombia           872
mauricio           776
francia            617
madagascar         459
malasia            453
paraguay           253
guatemala          237
serbia              96
lituania            79
bulgaria            78
mexico              73
tailandia           72
egipto              47
sin dato            38
portugal            35
corea del sur       19
italia               4
bielorusia           1
Name: Pais_Producto_fil, dtype: int64

## Filtrado de clientes que desde 2019 no realizan compras

El objetivo es filtrar la base de datos para obtener sólo los clientes activos a partir del 2020. 

Pasos:

1. Modificar columna colección para unificar escritura. Se reemplaza AW por FW

2. Filtrar clientes que no compran desde 2019. Para esto:
    1. se filtran las colecciones FW y SS desde 2020
    2. calculo de los clientes activos (es decir que han realizado compras desde 2020)
    3. Filtrado de dataset total para mantener sólo clientes activos (se mantienen compras desde 2016 pero sólo de los clientes activos)

In [33]:
# función para reemplazar 'AW' por 'FW'
def standardize_coleccion_name(name):
    return name.replace('AW', 'FW')

# Aplicar la función a la columna 'Coleccion'
data_proces_status['Coleccion'] = data_proces_status['Coleccion'].apply(standardize_coleccion_name)

data_proces_status['Coleccion'].value_counts()

/tmp/ipykernel_1064/1125347918.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_proces_status['Coleccion'] = data_proces_status['Coleccion'].apply(standardize_coleccion_name)


FW2018    6277
FW2016    4096
SS2024    3860
SS2018    3550
FW2024    3433
SS2019    3414
SS2016    3323
FW2017    3115
FW2022    2911
SS2017    2752
SS2023    2344
FW2023    2337
FW2019    2133
SS2020    2076
FW2020    1891
SS2022    1591
FW2021    1041
SS2021     621
Name: Coleccion, dtype: int64

In [34]:
# Identificar las colecciones válidas desde SS2020 en adelante
colecciones_validas = data_proces_status['Coleccion'].unique()
colecciones_validas

array(['FW2016', 'FW2017', 'FW2018', 'FW2019', 'FW2020', 'FW2021',
       'FW2022', 'FW2023', 'FW2024', 'SS2016', 'SS2017', 'SS2018',
       'SS2019', 'SS2020', 'SS2021', 'SS2022', 'SS2023', 'SS2024'],
      dtype=object)

In [35]:
# Comprenhesion list donde se pasa a número los últimos dos caracteres de la columna Colección para filtrar las temporadas desde 2020
colecciones_validas = [col for col in colecciones_validas if int(col[4:]) >= 20]

In [36]:
colecciones_validas

['FW2020',
 'FW2021',
 'FW2022',
 'FW2023',
 'FW2024',
 'SS2020',
 'SS2021',
 'SS2022',
 'SS2023',
 'SS2024']

In [37]:
# máscara booleana para las filas que tienen colecciones válidas
mask = data_proces_status['Coleccion'].isin(colecciones_validas)

In [38]:
# DataFrame para obtener solo las filas con colecciones válidas
data_df_filtered = data_proces_status[mask]

In [39]:
data_df_filtered

,Coleccion,Referencia,Descripcion,Color,No.Pedido,Tot_Piezas,Costo_Fabrica,Precio_Venta,Pais_Producto,Cta_Cliente,Nombre_Cliente,Estatus,Plataforma,familia,Pais_Producto_fil
18907,FW2020,RF6002,MASCARILLA DE USO NO QUIRURGICO,031,1,100.0,3.85,7.0,peru,2000006,RES.CLIENTES INTERNACIONALES,EN SISTEMA,,RF,peru
18908,FW2020,RF6101,MASCARILLA DE USO NO QUIRURGICO,LAW,1,100.0,10.26,18.0,peru,2000006,RES.CLIENTES INTERNACIONALES,EN SISTEMA,,RF,peru
18909,FW2020,RF6103,MASCARILLA DE USO NO QUIRURGICO,LAW,1,100.0,10.26,18.0,francia,2000006,RES.CLIENTES INTERNACIONALES,EN SISTEMA,,RF,francia
18910,FW2020,RF6123,MASCARILLA DE USO NO QUIRURGICO,2BE,1,41.0,8.11,14.0,francia,2000006,RES.CLIENTES INTERNACIONALES,EN SISTEMA,,RF,francia
18921,FW2020,RK2321,GORRA PARA CABALLERO,031,7859,24.0,15.65,26.0,china,1307082,"INVERSIONES GABYN,S.A.",EN SISTEMA,ASIA,RK,china
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65592,SS2024,TJ7951 PE,SUETER CUELLO REDONDO PARA NINO,001,9420,48.0,16.26,23.0,peru,1307012,RES.GABYN,02/19/2024,AMERICA,TJ,peru
65593,SS2024,TJ7951 PE,SUETER CUELLO REDONDO PARA NINO,001,9427,9.0,16.26,23.0,peru,2224074,ICOSAL S.A. DE C.V.,02/19/2024,AMERICA,TJ,peru
65594,SS2024,TJ7951 PE,SUETER CUELLO REDONDO PARA NINO,001,9430,6.0,16.26,23.0,peru,2224074,ICOSAL S.A. DE C.V.,02/19/2024,AMERICA,TJ,peru
65595,SS2024,TJ7951 PE,SUETER CUELLO REDONDO PARA NINO,001,9431,16.0,16.26,23.0,peru,2313076,"RIPOSTO,S.A.",02/19/2024,AMERICA,TJ,peru


In [40]:
# Obtener una lista de clientes activos (que han comprado desde SS2019)
clientes_activos = data_df_filtered['Nombre_Cliente'].unique()

In [41]:
clientes_activos

array(['RES.CLIENTES INTERNACIONALES', 'INVERSIONES GABYN,S.A.',
       'RIPOSTO,S.A.', 'CELL STAR DA VINCI ENTERPRICES',
       'ALMACEN JERUSALEN', 'RES.SUCURSALES SR.JACK',
       'ICOSAL S.A. DE C.V.', 'ADN GROUP, S.A.',
       'INVERSIONES MONTIJO, S.A.', 'REMOTEXBO, S.A.',
       'DISTRIBUIDORA ATLANTIS', 'DISTRIBUIDORA RIMET HONDURAS, S.A.',
       'ICOSAL,S.A. AEROPUERT', 'OPTIKA, S.A.', 'CITY MALL (DAVID)',
       '                                        ', 'RES. GABYN S.A.',
       'CORP.DE FRANQUICIAS CENTRO AMERICANA,S.A',
       'INVERSIONES Y REPRES. SALAZAR 2015,C.A.', 'H&D COMPANY S.R.L',
       'TIENDA DICONS SRL (SANTIAGO)', 'RESERVA GABYN',
       'DISTRIBUIDORA 2224', 'JOSE N. BATARSE, S.A. DE C.V.',
       'RES. GABYN', 'DISTRIBUIDORA 2224, C.A.', 'PIU MEIS S.R.L.',
       'BENJAMIN ALEXIS MATUTE ESCOBAR-LAC.', 'TIENDA PALM',
       'CELL STAR DA VINCI ENTERPRISES',
       'YOUNG FASHION  RAMON HIPOLITO PEREZ',
       'RES. CLIENTES INTERNACIONALES', 'RES.GABYN(2)'

In [42]:
data_df_filtered['Coleccion'].value_counts()

SS2024    3860
FW2024    3433
FW2022    2911
SS2023    2344
FW2023    2337
SS2020    2076
FW2020    1891
SS2022    1591
FW2021    1041
SS2021     621
Name: Coleccion, dtype: int64

In [43]:
# Filtrar el DataFrame original para mantener solo los registros de clientes activos
data_df_final = data_proces_status[data_proces_status['Nombre_Cliente'].isin(clientes_activos)]

In [44]:
data_df_final['Coleccion'].value_counts()

SS2024    3860
FW2018    3560
FW2024    3433
FW2022    2911
SS2023    2344
FW2023    2337
SS2020    2076
FW2020    1891
FW2019    1826
SS2019    1621
SS2022    1591
SS2018    1408
FW2016    1349
FW2017    1097
FW2021    1041
SS2017     850
SS2016     822
SS2021     621
Name: Coleccion, dtype: int64

In [45]:
data_df_final.shape

(34638, 15)

In [46]:
# valores únicos de la columna 'Nombre_Cliente' en el DataFrame filtrado
clientes_restantes = data_df_final['Nombre_Cliente'].unique()

# Convertir a lista para mejor visualización 
lista_clientes_restantes = list(clientes_restantes)

lista_clientes_restantes

['RIPOSTO,S.A.',
 'ICOSAL S.A. DE C.V.',
 'ICOSAL,S.A. AEROPUERT',
 'INVERSIONES MONTIJO, S.A.',
 'REMOTEXBO, S.A.',
 'OPTIKA, S.A.',
 'RES.CLIENTES INTERNACIONALES',
 'ALMACEN JERUSALEN',
 'RES. CLIENTES INTERNACIONALES',
 'CORP.DE FRANQUICIAS CENTRO AMERICANA,S.A',
 'RES.GABYN',
 'RES.SUCURSALES SR.JACK',
 'DISTRIBUIDORA RIMET HONDURAS, S.A.',
 'DISTRIBUIDORA ATLANTIS',
 'CELL STAR DA VINCI ENTERPRICES',
 'CITY MALL (DAVID)',
 'INVERSIONES GABYN,S.A.',
 'ADN GROUP, S.A.',
 '                                        ',
 'RES. GABYN S.A.',
 'INVERSIONES Y REPRES. SALAZAR 2015,C.A.',
 'H&D COMPANY S.R.L',
 'TIENDA DICONS SRL (SANTIAGO)',
 'RESERVA GABYN',
 'DISTRIBUIDORA 2224',
 'JOSE N. BATARSE, S.A. DE C.V.',
 'RES. GABYN',
 'DISTRIBUIDORA 2224, C.A.',
 'PIU MEIS S.R.L.',
 'BENJAMIN ALEXIS MATUTE ESCOBAR-LAC.',
 'TIENDA PALM',
 'CELL STAR DA VINCI ENTERPRISES',
 'YOUNG FASHION  RAMON HIPOLITO PEREZ',
 'RES.GABYN(2)',
 'ANDRES GOMEZ FASHION STORE SRL',
 'TIENDA DICONS SRL (LA VEGA)',
 'P

In [47]:
len(lista_clientes_restantes)

37

In [48]:
data_df_final['Nombre_Cliente'].value_counts()

RES.SUCURSALES SR.JACK                      3591
RIPOSTO,S.A.                                3527
ICOSAL S.A. DE C.V.                         3291
INVERSIONES MONTIJO, S.A.                   3218
REMOTEXBO, S.A.                             1960
RES.CLIENTES INTERNACIONALES                1959
DISTRIBUIDORA ATLANTIS                      1885
ALMACEN JERUSALEN                           1834
DISTRIBUIDORA RIMET HONDURAS, S.A.          1610
CORP.DE FRANQUICIAS CENTRO AMERICANA,S.A    1318
ICOSAL,S.A. AEROPUERT                       1193
CELL STAR DA VINCI ENTERPRICES              1103
PIU MEIS S.R.L.                              825
RES.GABYN                                    811
DISTRIBUIDORA 2224, C.A.                     782
OPTIKA, S.A.                                 758
CITY MALL (DAVID)                            606
JOSE N. BATARSE, S.A. DE C.V.                587
RESERVA GABYN                                583
RES. GABYN S.A.                              503
TIENDA PALM         

### Ajustes columnas "Cliente"

In [ ]:
# Verificar cliente en blanco, cadena vacia
compras_cliente_blanco = data_df_final[data_df_final['Nombre_Cliente'] == '']

In [78]:
# REVISAR CÓDIGO PORQUE NO ESTÁ TOMANDO LOS CLIENTES EN BLANCO
compras_cliente_blanco

,Coleccion,Referencia,Descripcion,Color,No.Pedido,Tot_Piezas,Costo_Fabrica,Precio_Venta,Pais_Producto,Cta_Cliente,Nombre_Cliente,Estatus,Plataforma,familia,Pais_Producto_fil


**REVISAR DE ACÁ PARA ADELANTE**

In [ ]:
# piezas compradas por el cliente en blanco
total_piezas_cliente_blanco = compras_cliente_blanco['Tot_Piezas'].sum()
total_piezas_cliente_blanco

In [ ]:
# Elimina las filas donde 'Nombre_Cliente' es una cadena vacía o solo contiene espacios en blanco
data_df_final_sin_blanco = data_df_final[data_df_final['Nombre_Cliente'].str.strip() != '']

data_df_final_sin_blanco.head()

In [ ]:
data_df_final_sin_blanco['Estatus'].value_counts()

In [ ]:
data_df_final_sin_blanco['Pais_Producto_fil'].value_counts()

In [ ]:
data_df_final_sin_blanco['Nombre_Cliente'].value_counts()

In [ ]:
data_df_final_sin_blanco.head()

In [32]:
data_proces_status.head()

,Coleccion,Referencia,Descripcion,Color,No.Pedido,Tot_Piezas,Costo_Fabrica,Precio_Venta,Pais_Producto,Cta_Cliente,Nombre_Cliente,Estatus,Plataforma,familia,Pais_Producto_fil
0,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5590,12.0,15.65,25.0,china,1207097,RESERVA SUCURSALES,EN SISTEMA,ASIA,RK,china
1,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5595,12.0,15.65,25.0,china,1207097,RESERVA SUCURSALES,EN SISTEMA,ASIA,RK,china
2,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5597,6.0,15.65,25.0,china,2313076,"RIPOSTO,S.A.",EN SISTEMA,ASIA,RK,china
3,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5599,4.0,15.65,25.0,china,2305201,"MYL DE COLOMBIA,SAS",EN SISTEMA,ASIA,RK,china
4,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5600,4.0,15.65,25.0,china,2305201,"MYL DE COLOMBIA,SAS",EN SISTEMA,ASIA,RK,china


In [33]:
len(valores_unicos['Nombre_Cliente'])

75

In [34]:
len(valores_unicos['Cta_Cliente'])

55

In [38]:
data_proces_status['Nombre_Cliente'].value_counts()

RES.SUCURSALES SR.JACK                3591
RIPOSTO,S.A.                          3527
ICOSAL S.A. DE C.V.                   3291
INVERSIONES MONTIJO, S.A.             3218
MYL DE COLOMBIA,SAS                   2844
                                      ... 
IMPERIA 01                               9
ELVIS JUNIOR SRL                         8
SONISAM SPORT CIA. LTDA                  4
ALMACENES FRANCINI S.A.S.                3
IDAGO LIMITADA (OMAR IDARRAGA OSM)       1
Name: Nombre_Cliente, Length: 75, dtype: int64

In [36]:
data_proces_status['Cta_Cliente'].value_counts()

1207097    4797
2313076    3527
2224074    3291
2209051    3218
2305201    2844
1307012    2405
2306225    1960
2214102    1885
2220157    1834
2000006    1757
2403005    1746
2214097    1610
2316047    1589
2218028    1511
1307011    1325
2224090    1193
2305196    1173
2305178    1022
2327376     847
2201119     825
2319052     809
2601001     758
1307082     753
2319071     685
0           661
2305188     645
2319046     607
2220239     606
2224094     587
2305222     497
2325025     495
2214066     429
2305185     428
2218024     330
2401117     222
2305219     218
2319077     208
2218018     201
2220233     187
2401112     149
2313074     147
2326028     107
2319066     103
1220055      86
2327373      84
2401116      78
2319051      69
2401118      69
2214054      59
2401114      51
2327377      38
2214118      19
1207086       9
2401100       8
2312112       4
Name: Cta_Cliente, dtype: int64

In [46]:
# seleccionar los clientes con la cuenta número 1207097
cliente = data_proces_status.loc[:,'Cta_Cliente'] == 1207097
df_cliente = data_proces_status.loc[cliente]
#pd.options.display.max_rows = None #código para que muestre todas las filas
df_cliente.head()

,Coleccion,Referencia,Descripcion,Color,No.Pedido,Tot_Piezas,Costo_Fabrica,Precio_Venta,Pais_Producto,Cta_Cliente,Nombre_Cliente,Estatus,Plataforma,familia,Pais_Producto_fil
0,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5590,12.0,15.65,25.0,china,1207097,RESERVA SUCURSALES,EN SISTEMA,ASIA,RK,china
1,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,001,5595,12.0,15.65,25.0,china,1207097,RESERVA SUCURSALES,EN SISTEMA,ASIA,RK,china
16,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,031,5590,12.0,15.65,25.0,china,1207097,RESERVA SUCURSALES,EN SISTEMA,ASIA,RK,china
17,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,031,5595,12.0,15.65,25.0,china,1207097,RESERVA SUCURSALES,EN SISTEMA,ASIA,RK,china
32,AW2016,RK2321,GORRA LACOSTE PARA CABALLERO,166,5590,12.0,15.65,25.0,china,1207097,RESERVA SUCURSALES,EN SISTEMA,ASIA,RK,china


In [45]:
# valores únicos para la cuenta cliente = 1207097
df_cliente['Nombre_Cliente'].value_counts()

RES.SUCURSALES SR.JACK                      3591
RESERVA SUCURSALES                           723
GABYSOL,S.A.                                 242
RESERVAS SUCURSALES                          224
                                              16
RES.GABYN(2)                                   1
Name: Nombre_Cliente, dtype: int64

Pasos:

1° duplicados, NaN, filtrar "cancelados", filtrar clientes que desde 2019 no compran

2° Colección: SS -- 01-05-2016   -   FW -- 01-11-2016
             Agregar 2 nueva columna SS o FW
   Referencia -- Familia -- "encoding" (agregar columnas)
   Total piezas (y - target)
   país productor -- corregir texto con distinta escritura (Servia, Camboya) 
                  -- encoding
   Cliente -- encoding
   Plataforma -- encoding

función para reemplazar 'AW' con 'FW'
def standardize_coleccion_name(name):
    return name.replace('AW', 'FW')

Aplicar la función a la columna 'Coleccion'
data_proces_status['Coleccion'] = data_proces_status['Coleccion'].apply(standardize_coleccion_name)

data_proces_status['Coleccion'].value_counts()